# Calculating Economic Impact Using Crop Data

In this Python notebook, we will explore how to calculate the economic impact of crop production using crop data. Agriculture plays a crucial role in the economy of many countries, and understanding the economic implications of crop production is essential for policymakers, farmers, and agricultural researchers.

We will cover the following key aspects in this notebook:

1. Data Collection: We will begin by gathering relevant crop data. This can include information on crop yields and acreage

2. Data Preprocessing: Data preprocessing is a critical step to ensure that the data is clean and ready for analysis. We will clean, format, and organize the data as needed.

3. Economic Impact Analysis: We will perform various calculations to assess the economic impact of crop production. This may involve calculating the gross revenue generated by crops, net income after deducting production costs, and economic multipliers that account for the ripple effects of agricultural spending in the local economy.

## Loading Data

In [1]:
# importing libraries

import pandas as pd

In [2]:
# reading the original file

df = pd.read_csv("data/all-crops-data.csv")

/var/folders/9r/7q8l4tjd43z8stsvm3s_5g400000gn/T/ipykernel_26434/2709926224.py:3: DtypeWarning: Columns (25,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/all-crops-data.csv")


In [3]:
# viewing columns

df.columns

Index(['SOURCE_DESC', 'SECTOR_DESC', 'GROUP_DESC', 'COMMODITY_DESC',
       'CLASS_DESC', 'PRODN_PRACTICE_DESC', 'UTIL_PRACTICE_DESC',
       'STATISTICCAT_DESC', 'UNIT_DESC', 'SHORT_DESC', 'DOMAIN_DESC',
       'DOMAINCAT_DESC', 'AGG_LEVEL_DESC', 'STATE_ANSI', 'STATE_FIPS_CODE',
       'STATE_ALPHA', 'STATE_NAME', 'ASD_CODE', 'ASD_DESC', 'COUNTY_ANSI',
       'COUNTY_CODE', 'COUNTY_NAME', 'REGION_DESC', 'ZIP_5', 'WATERSHED_CODE',
       'WATERSHED_DESC', 'CONGR_DISTRICT_CODE', 'COUNTRY_CODE', 'COUNTRY_NAME',
       'LOCATION_DESC', 'YEAR', 'FREQ_DESC', 'BEGIN_CODE', 'END_CODE',
       'REFERENCE_PERIOD_DESC', 'WEEK_ENDING', 'LOAD_TIME', 'VALUE', 'CV_%'],
      dtype='object')

In [4]:
# filtering columns

df_filtered = df[['COMMODITY_DESC', 'UNIT_DESC', 'YEAR', 'STATE_NAME', 'COUNTY_NAME', 'STATISTICCAT_DESC', 'VALUE']]

In [5]:
# keeping only Muskogee, Oklahoma data

df_okl = df_filtered[df_filtered['STATE_NAME'] == 'OKLAHOMA']
df_okl = df_filtered[df_filtered['COUNTY_NAME'] == 'MUSKOGEE']

In [6]:
# elimination of rows that have the value of '(D)'
# (D) = Withheld to avoid disclosing data for individual operations

df_okl = df_okl[df_okl['VALUE'] != '(D)']

In [7]:
# viewing the final dataset
df_okl = df_okl.reset_index(drop = True)
df_okl.head(10)

,COMMODITY_DESC,UNIT_DESC,YEAR,STATE_NAME,COUNTY_NAME,STATISTICCAT_DESC,VALUE
0,HAYLAGE,TONS,2017,OKLAHOMA,MUSKOGEE,PRODUCTION,"2,023"
1,WHEAT,ACRES,1963,OKLAHOMA,MUSKOGEE,AREA HARVESTED,"7,100"
2,WHEAT,ACRES,1981,OKLAHOMA,MUSKOGEE,AREA HARVESTED,"31,200"
3,COTTON,LB / ACRE,1975,OKLAHOMA,MUSKOGEE,YIELD,359
4,HAY,ACRES,1978,OKLAHOMA,MUSKOGEE,AREA HARVESTED,"38,100"
5,BARLEY,BU / ACRE,1971,OKLAHOMA,MUSKOGEE,YIELD,28.2
6,WHEAT,BU / ACRE,1925,OKLAHOMA,MUSKOGEE,YIELD,7.6
7,WHEAT,BU,2005,OKLAHOMA,MUSKOGEE,PRODUCTION,"221,000"
8,CORN,ACRES,1987,OKLAHOMA,MUSKOGEE,AREA HARVESTED,"2,500"
9,BLACKBERRIES,OPERATIONS,2007,OKLAHOMA,MUSKOGEE,AREA HARVESTED,6


In [8]:
# filtering data to include only the last 60 years of data

df = df_okl.copy()
df = df[df['YEAR'] >= 1963]
df.sort_values(by = 'YEAR', ascending = True, inplace = True)

In [9]:
df['STATISTICCAT_DESC'].unique()

array(['PRODUCTION', 'AREA HARVESTED', 'YIELD', 'AREA PLANTED',
       'AREA PLANTED, NET', 'AREA BEARING & NON-BEARING', 'SALES',
       'AREA IN PRODUCTION', 'AREA NON-BEARING', 'CAPACITY',
       'AREA BEARING', 'AREA GROWN', 'AREA NOT HARVESTED', 'OPERATIONS'],
      dtype=object)

In [10]:
df['COMMODITY_DESC'].unique()

array(['SORGHUM', 'BARLEY', 'HAY', 'OATS', 'WHEAT', 'SOYBEANS', 'CORN',
       'COTTON', 'PEANUTS', 'RYE', 'ORCHARDS', 'HAYLAGE', 'CROP TOTALS',
       'BLACKBERRIES', 'BEDDING PLANT TOTALS', 'BLUEBERRIES',
       'VEGETABLE TOTALS', 'BERRY TOTALS', 'SPINACH',
       'FRUIT & TREE NUT TOTALS', 'GREENS', 'PEAS', 'HORTICULTURE TOTALS',
       'PLUMS & PRUNES', 'PEPPERS', 'GRAIN STORAGE CAPACITY',
       'TREE NUTS, OTHER', 'FLORICULTURE TOTALS', 'CUT CHRISTMAS TREES',
       'HAY & HAYLAGE', 'PECANS', 'PEACHES', 'SWEET CORN', 'APPLES',
       'CUT FLOWERS & CUT CULTIVATED GREENS', 'TOMATOES',
       'FIELD CROPS, OTHER', 'MELONS', 'FOLIAGE PLANTS', 'GRAIN',
       'STRAWBERRIES', 'OKRA', 'FLOWERING PLANTS, POTTED', 'BEANS',
       'PUMPKINS', 'CUT CHRISTMAS TREES & SHORT TERM WOODY CROPS',
       'NURSERY TOTALS', 'GRAPES', 'CUCUMBERS', 'SOD',
       'NON-CITRUS TOTALS', 'TREE NUT TOTALS', 'POTATOES', 'LEGUMES',
       'GRASSES & LEGUMES TOTALS', 'ALMONDS', 'SQUASH', 'CHERRIES',
       '

In [11]:
df = df[df['STATISTICCAT_DESC'].isin(['AREA PLANTED', 'AREA HARVESTED'])]

In [12]:
# calculate the counts for each unique value in 'COMMODITY_DESC'
counts = df['COMMODITY_DESC'].value_counts()

# use transform to broadcast the counts back to the original DataFrame
df['COMMODITY_DESC_COUNT'] = df['COMMODITY_DESC'].map(counts)

# filter rows where the count is greater than or equal to 60 (to get data for 60 years)
df = df[df['COMMODITY_DESC_COUNT'] >= 60]

# drop the count column if you don't need it anymore
df = df.drop(columns = ['COMMODITY_DESC_COUNT'], axis = 1)

In [13]:
df['COMMODITY_DESC'].value_counts()

WHEAT       429
HAY         210
SOYBEANS    163
CORN        163
SORGHUM     147
OATS         86
COTTON       63
Name: COMMODITY_DESC, dtype: int64

In [14]:
df = df.drop(columns = ['STATE_NAME', 'COUNTY_NAME'], axis = 1).reset_index(drop = True)

In [15]:
df.head(10)

,COMMODITY_DESC,UNIT_DESC,YEAR,STATISTICCAT_DESC,VALUE
0,HAY,ACRES,1963,AREA HARVESTED,"26,300"
1,WHEAT,ACRES,1963,AREA HARVESTED,"7,100"
2,SOYBEANS,ACRES,1963,AREA PLANTED,"18,800"
3,OATS,ACRES,1963,AREA PLANTED,"5,300"
4,CORN,ACRES,1963,AREA HARVESTED,"4,100"
5,SOYBEANS,ACRES,1963,AREA HARVESTED,"16,200"
6,SORGHUM,ACRES,1963,AREA PLANTED,"9,000"
7,COTTON,ACRES,1963,AREA PLANTED,"9,520"
8,HAY,ACRES,1963,AREA HARVESTED,"29,600"
9,SORGHUM,ACRES,1963,AREA HARVESTED,"6,300"


In [16]:
# saving the muskogee crops dataset

df.to_csv("data/muskogee-crops.csv")

## Agregating Crops to Yearly Data

In [17]:
# group by the year and getting the sum

df_yearly = df.groupby(['COMMODITY_DESC', 'UNIT_DESC', 'YEAR', 'STATISTICCAT_DESC']).sum().reset_index()
df_yearly.head(10)

,COMMODITY_DESC,UNIT_DESC,YEAR,STATISTICCAT_DESC,VALUE
0,CORN,ACRES,1963,AREA HARVESTED,"4,100"
1,CORN,ACRES,1963,AREA PLANTED,"4,600"
2,CORN,ACRES,1964,AREA HARVESTED,"2,800"
3,CORN,ACRES,1964,AREA PLANTED,"3,350"
4,CORN,ACRES,1965,AREA HARVESTED,"2,200"
5,CORN,ACRES,1965,AREA PLANTED,"2,400"
6,CORN,ACRES,1966,AREA HARVESTED,"1,300"
7,CORN,ACRES,1966,AREA PLANTED,"2,000"
8,CORN,ACRES,1967,AREA HARVESTED,800
9,CORN,ACRES,1967,AREA PLANTED,"1,200"


In [18]:
df_yearly.shape

(620, 5)

In [19]:
# transforming the dataset into a wide dataset

df_pivot = df_yearly.pivot(index=['COMMODITY_DESC', 'UNIT_DESC', 'YEAR'], columns='STATISTICCAT_DESC', values='VALUE').reset_index()

In [20]:
df_pivot.head(15)

STATISTICCAT_DESC,COMMODITY_DESC,UNIT_DESC,YEAR,AREA HARVESTED,AREA PLANTED
0,CORN,ACRES,1963,"4,100","4,600"
1,CORN,ACRES,1964,"2,800","3,350"
2,CORN,ACRES,1965,"2,200","2,400"
3,CORN,ACRES,1966,"1,300","2,000"
4,CORN,ACRES,1967,800,"1,200"
5,CORN,ACRES,1968,900,"1,100"
6,CORN,ACRES,1969,"1,100","1,500"
7,CORN,ACRES,1970,"1,200","1,400"
8,CORN,ACRES,1971,"1,200","1,500"
9,CORN,ACRES,1972,"1,700","1,800"


In [21]:
# checking for null values

df_pivot.isna().sum()

STATISTICCAT_DESC
COMMODITY_DESC     0
UNIT_DESC          0
YEAR               0
AREA HARVESTED     0
AREA PLANTED      90
dtype: int64

In [22]:
# dropping null values

df_pivot = df_pivot.dropna()

In [23]:
df_pivot['COMMODITY_DESC'].value_counts()

SOYBEANS    59
CORN        57
WHEAT       54
SORGHUM     44
OATS        35
COTTON      16
Name: COMMODITY_DESC, dtype: int64

In [24]:
df_pivot.head(10)

STATISTICCAT_DESC,COMMODITY_DESC,UNIT_DESC,YEAR,AREA HARVESTED,AREA PLANTED
0,CORN,ACRES,1963,"4,100","4,600"
1,CORN,ACRES,1964,"2,800","3,350"
2,CORN,ACRES,1965,"2,200","2,400"
3,CORN,ACRES,1966,"1,300","2,000"
4,CORN,ACRES,1967,800,"1,200"
5,CORN,ACRES,1968,900,"1,100"
6,CORN,ACRES,1969,"1,100","1,500"
7,CORN,ACRES,1970,"1,200","1,400"
8,CORN,ACRES,1971,"1,200","1,500"
9,CORN,ACRES,1972,"1,700","1,800"


# Crop Impact Score

The formula used to calculate the impact of wildfires on crop production is as follows. 

**crop impact score = ((max(area harvested) / max(area planted)) - (area harvested / area planted)) / (max(area harvested) / max(area planted))**

This formula computes a "CROP IMPACT" score for each row in a DataFrame (presumably containing data related to crop production) based on the relative change in the harvested area compared to a reference point. Here's a step-by-step explanation of how the formula works:

1. **Area Harvested %**: First step is to calculate the area harvested percentage for all the rows

2. **Crop Impact**: This is the main calculation of the crop impact. It calculates the relative difference between the max area harvested percentage and the area harvested percentage for that particular year. The formula subtracts the harvested area percentage of each crop from the maximum harvested area percentage, and then divides the result by the maximum area percentage. Finally, this result is assigned to a new column 'CROP IMPACT' in the DataFrame.

The rationale behind this formula is to quantify the impact of wildfires on crop production by comparing the harvested area of each crop to a reference crop (the one with the largest harvested area). If a crop's harvested area is similar to the reference crop, the 'CROP IMPACT' score will be close to zero, indicating little impact. On the other hand, if a crop's harvested area is significantly lower than the reference crop, the 'CROP IMPACT' score will be closer to one, indicating a more severe impact.

This formula allows for a relative assessment of how different crops are affected by wildfires, providing a numeric measure that can be used for analysis, visualization, and decision-making related to crop management and disaster response.

In [25]:
# integrating the crop impact formula

df = df_pivot.copy()

# filter the DataFrame to keep rows with values of 6 characters or less (as there are values that are too long and not accurate)
df = df[df['AREA HARVESTED'].str.len() <= 6]

df.head(15)

STATISTICCAT_DESC,COMMODITY_DESC,UNIT_DESC,YEAR,AREA HARVESTED,AREA PLANTED
0,CORN,ACRES,1963,"4,100","4,600"
1,CORN,ACRES,1964,"2,800","3,350"
2,CORN,ACRES,1965,"2,200","2,400"
3,CORN,ACRES,1966,"1,300","2,000"
4,CORN,ACRES,1967,800,"1,200"
5,CORN,ACRES,1968,900,"1,100"
6,CORN,ACRES,1969,"1,100","1,500"
7,CORN,ACRES,1970,"1,200","1,400"
8,CORN,ACRES,1971,"1,200","1,500"
9,CORN,ACRES,1972,"1,700","1,800"


In [26]:
# replace commas and convert to integers
df['AREA HARVESTED'] = df['AREA HARVESTED'].str.replace(',', '').astype('float64')
df['AREA PLANTED'] = df['AREA PLANTED'].str.replace(',', '').astype('float64')

df = df.sort_values(by='AREA HARVESTED', ascending=False)
df.head(15)

STATISTICCAT_DESC,COMMODITY_DESC,UNIT_DESC,YEAR,AREA HARVESTED,AREA PLANTED
224,SORGHUM,ACRES,2007,717800.0,1100.0
174,OATS,ACRES,2007,300204.0,1100.0
245,SOYBEANS,ACRES,1977,37500.0,39000.0
246,SOYBEANS,ACRES,1978,34500.0,37500.0
247,SOYBEANS,ACRES,1979,34000.0,38000.0
257,SOYBEANS,ACRES,1989,33800.0,35000.0
244,SOYBEANS,ACRES,1976,29500.0,30000.0
248,SOYBEANS,ACRES,1980,29500.0,34000.0
256,SOYBEANS,ACRES,1988,29000.0,30000.0
250,SOYBEANS,ACRES,1982,27500.0,29000.0


We can see that there are two values at the top which are inaccurate so we filter the dataset to remove those rows

In [27]:
df = df[df['AREA HARVESTED'] <= 100000]
df.head(15)

STATISTICCAT_DESC,COMMODITY_DESC,UNIT_DESC,YEAR,AREA HARVESTED,AREA PLANTED
245,SOYBEANS,ACRES,1977,37500.0,39000.0
246,SOYBEANS,ACRES,1978,34500.0,37500.0
247,SOYBEANS,ACRES,1979,34000.0,38000.0
257,SOYBEANS,ACRES,1989,33800.0,35000.0
244,SOYBEANS,ACRES,1976,29500.0,30000.0
248,SOYBEANS,ACRES,1980,29500.0,34000.0
256,SOYBEANS,ACRES,1988,29000.0,30000.0
250,SOYBEANS,ACRES,1982,27500.0,29000.0
237,SOYBEANS,ACRES,1969,26600.0,29300.0
258,SOYBEANS,ACRES,1990,25900.0,29800.0


In [28]:
df.dtypes

STATISTICCAT_DESC
COMMODITY_DESC     object
UNIT_DESC          object
YEAR                int64
AREA HARVESTED    float64
AREA PLANTED      float64
dtype: object

In [29]:
df['AREA HARVESTED %'] = df['AREA HARVESTED'] / df['AREA PLANTED']

# step 1: Group by "COMMODITY_DESC" and calculate the maximum area harvested percentage for each crop type
max_area_harvested_percent = df.groupby('COMMODITY_DESC')['AREA HARVESTED %'].max().reset_index()

# step 2: Merge the maximum values back into the original DataFrame based on "COMMODITY_DESC"
df = df.merge(max_area_harvested_percent, on='COMMODITY_DESC', suffixes=('', '_MAX'))

# step 3: Calculate the "CROP IMPACT" using the formula
df['CROP IMPACT'] = (df['AREA HARVESTED %_MAX'] - df['AREA HARVESTED %']) / df['AREA HARVESTED %_MAX']

df_final = df.drop(['AREA HARVESTED %_MAX', 'AREA HARVESTED %'], axis = 1)

In [30]:
df_final.head(15)

,COMMODITY_DESC,UNIT_DESC,YEAR,AREA HARVESTED,AREA PLANTED,CROP IMPACT
0,SOYBEANS,ACRES,1977,37500.0,39000.0,0.038462
1,SOYBEANS,ACRES,1978,34500.0,37500.0,0.080000
2,SOYBEANS,ACRES,1979,34000.0,38000.0,0.105263
3,SOYBEANS,ACRES,1989,33800.0,35000.0,0.034286
4,SOYBEANS,ACRES,1976,29500.0,30000.0,0.016667
5,SOYBEANS,ACRES,1980,29500.0,34000.0,0.132353
6,SOYBEANS,ACRES,1988,29000.0,30000.0,0.033333
7,SOYBEANS,ACRES,1982,27500.0,29000.0,0.051724
8,SOYBEANS,ACRES,1969,26600.0,29300.0,0.092150
9,SOYBEANS,ACRES,1990,25900.0,29800.0,0.130872


In [31]:
df_final.to_csv("data/harvest-muskogee.csv", index = False)